# Portfolio aggregations
In this notebook we'll give some examples on how the portfolio aggregation methods can be used.

### Setting up
We'll set up the imports, data providers, and load the portfolio. For more details about this, please have a look at notebook one (getting started).

In [ ]:
# Only required if you didn't install the module yet
!pip install SBTi

In [1]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
%aimport -pandas
import pandas as pd

In [2]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/example_portfolio.csv", "example_portfolio.csv")

In [15]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")
portfolio = pd.read_csv("example_portfolio.csv", encoding="iso-8859-1")
companies = [SBTi.interfaces.PortfolioCompany.parse_obj(company) for company in portfolio.to_dict(orient="records")]
portfolio_data = SBTi.utils.get_data([provider], companies)

In [16]:
temperature_score = TemperatureScore(time_frames=list(SBTi.interfaces.ETimeFrames), scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3])
amended_portfolio = temperature_score.calculate(portfolio_data)

### Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done use different aggregation methods. The termperature scores are calculated per time-frame/scope combination.

In [17]:
def print_aggregations(aggregations):
    aggregations = aggregations.dict()
    for time_frame, time_frame_values in aggregations.items():
        for scope, scope_values in time_frame_values.items():
            print("{} - {}: {} degrees celcius".format(time_frame, scope, scope_values["all"]["score"]))

#### WATS
Weighted Average Temperature Score (WATS): Temperature scores are allocated based on portfolio weights.
This requires the "investment_value" field to be defined in your data set.

In [18]:
temperature_score.aggregation_method = PortfolioAggregationMethod.WATS
print_aggregations(temperature_score.aggregate_scores(amended_portfolio))

short - S1S2: 2.2349090909090905 degrees celcius
short - S3: 2.8205454545454542 degrees celcius
short - S1S2S3: 2.670272727272727 degrees celcius
mid - S1S2: 2.7351818181818177 degrees celcius
mid - S3: 3.0340000000000003 degrees celcius
mid - S1S2S3: 2.9639090909090906 degrees celcius
long - S1S2: 3.122181818181818 degrees celcius
long - S3: 3.2258181818181817 degrees celcius
long - S1S2S3: 3.199636363636363 degrees celcius


#### TETS
Total emissions weighted temperature score (TETS): Temperature scores are allocated based on historical emission weights using total company emissions.

In [ ]:
temperature_score.aggregation_method = PortfolioAggregationMethod.TETS
print_aggregations(temperature_score.aggregate_scores(amended_portfolio))

#### MOTS
Market Owned emissions weighted temperature score (MOTS): Temperature scores are allocated based on an equity ownership approach.

In [ ]:
temperature_score.aggregation_method = PortfolioAggregationMethod.MOTS
print_aggregations(temperature_score.aggregate_scores(amended_portfolio))

#### EOTS
Enterprise Owned emissions weighted temperature score (EOTS): Temperature scores are allocated based
on an enterprise ownership approach.

In [ ]:
temperature_score.aggregation_method = PortfolioAggregationMethod.EOTS
print_aggregations(temperature_score.aggregate_scores(amended_portfolio))

#### ECOTS
Enterprise Value + Cash emissions weighted temperature score (ECOTS): Temperature scores are allocated based on an enterprise value (EV) plus cash & equivalents ownership approach.

In [ ]:
temperature_score.aggregation_method = PortfolioAggregationMethod.ECOTS
print_aggregations(temperature_score.aggregate_scores(amended_portfolio))

#### AOTS
Total Assets emissions weighted temperature score (AOTS): Temperature scores are allocated based on a total assets ownership approach.

In [ ]:
temperature_score.aggregation_method = PortfolioAggregationMethod.AOTS
print_aggregations(temperature_score.aggregate_scores(amended_portfolio))